In [1]:
# Dependencies
from splinter import Browser 
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import requests
import pandas as pd
import os
executable_path = {"executable_path": "chromedriver.exe"}
browser = Browser("chrome", **executable_path, headless=False)

In [8]:
browser.visit(url)           
soup = bs(browser.html, 'html.parser')

In [9]:
#Retrieve page with the requests module, #get html
#response = requests.get(url)

In [10]:
# Create BeautifulSoup object; parse with 'html.parser'
#soup = bs(response.text, 'html.parser')

In [11]:
#Scape and print NASA latest news title
latest_news_title = soup.find('div',class_="content_title").text
print(f"Title:{latest_news_title}")

Title:Update on Opportunity Rover after Martian Dust Storm


In [12]:
# Scrape and print a paragraph text
news_para = soup.find('div', class_="article_teaser_body").text
print(f"Paragraph:{news_para}")

Paragraph:One month since increasing their commanding frequency, engineers have yet to hear from NASA's Opportunity rover.


# JPL Mars Space Images - Featured Image

In [22]:
# Featured image url page to be scraped
img_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
# Scrape and print the featured image url
browser.visit(img_url)           
browser.visit(img_url)           
soup = bs(browser.html, 'html.parser')
items = soup.find("div", class_="carousel_items")
img_title = items.find("h1",{"class":"media_feature_title"})
featured_img = items.find("article")
featured_img_url = img_url + featured_img['style'].split('\'')[1]
print(f"Featured Image:{featured_img_url}")

Featured Image:https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars/spaceimages/images/wallpaper/PIA18429-1920x1200.jpg


# Mars Weather

In [72]:
# Print mars weather's latest tweet from the Mars twitter account
weather_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(weather_url)       
twitter_soup = bs(browser.html, 'html.parser')
mars_tweets = twitter_soup.find_all('div', class_="js-tweet-text-container")
print(f"Weather Latest Tweet:{mars_tweets[0].text}")
for i in range(10):
    mars_weather = mars_tweets[i].text
    if "Sol " in mars_weather:
        print(f"Mars Weather:{mars_weather}")
        break       


Weather Latest Tweet:
And finally - a stack of all the images - you can see so much detail here https://dougellison.smugmug.com/Machines/Space-X-SAOCOM-1A/i-zbJzq8L …pic.twitter.com/9nLwAH3oZQ

Mars Weather:
Sol 2171 (2018-09-14), high -12C/10F, low -65C/-84F, pressure at 8.79 hPa, daylight 05:43-17:59



# Mars Facts

In [73]:
# Use Pandas to crap the mars facts table
facts_url = 'https://space-facts.com/mars/'
browser.visit(facts_url)
soup = bs(browser.html, 'html.parser')
fact_table = soup.find('table', class_='tablepress tablepress-id-mars')
column1 = fact_table.find_all('td', class_='column-1')
column2 = fact_table.find_all('td', class_='column-2')

features = []
values = []

for row in column1:
    feature = row.text.strip()
    features.append(feature)
    
for row in column2:
    value = row.text.strip()
    values.append(value)
    
mars_facts = pd.DataFrame({
    "Feature":features,
    "Value":values
    })

mars_facts.set_index("Feature")

,Value
Feature,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [79]:
mars_facts_html = mars_facts.to_html()
mars_facts_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Feature</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium BC<

# Mars Hemispheres

In [31]:
# Scrape images of Mars' hemispheres from the USGS site 
hemi_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemi_url)
hemi_soup = bs(browser.html, 'html.parser')
hemi_title = hemi_soup.find_all('a', class_='product-item')

# created a dictionary to store the data.
hemisphere_image_urls = []

# loop throught all titles,remove Enhanced, and print all titles
for t in hemi_title:
      
    title = t.text.strip('Enhanced')
    #print(title)
    
    img_link = "https://astrogeology.usgs.gov/" + t['href']
    #print(img_link)
    img_request = requests.get(img_link)
    img_soup = bs(img_request.text, 'lxml')
    img = img_soup.find('div', class_='downloads')
    img_path = img.find('a')['href']
    #print(img_path)
 
        
# Appending dictionary and print
    hemisphere_image_urls.append({"Title": title, "Image_Url": img_path})
    print(hemisphere_image_urls)

[{'Title': '', 'Image_Url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}]
[{'Title': '', 'Image_Url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'Title': 'Cerberus Hemisphere ', 'Image_Url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}]
[{'Title': '', 'Image_Url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'Title': 'Cerberus Hemisphere ', 'Image_Url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'Title': '', 'Image_Url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}]
[{'Title': '', 'Image_Url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'Title': 'Cerberus Hemisphere ', 'Image_Url': 'http://astropedia.astrogeology.usgs.gov/dow